In [1]:
import os

In [2]:
os.getcwd()

'C:\\Users\\srika\\Desktop\\Working Folder\\R\\Projects\\USIF'

In [3]:
os.chdir(r'C:\Users\srika\Desktop\Working Folder\R\Projects\USIF')

In [4]:
import pandas as pd
import numpy as np
import re
import nltk
import gensim
import string
import operator
from nltk.stem import WordNetLemmatizer
from gensim.models import word2vec
from gensim.models.keyedvectors import KeyedVectors
from gensim.models.phrases import Phrases, Phraser
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
tqdm.pandas()
lemmatizer = WordNetLemmatizer()
pd.set_option('display.max_colwidth', -1)
from gensim.utils import any2unicode
from numpy import float32 as REAL, sum as np_sum, multiply as np_mult
from fse import CSplitIndexedList
from fse.models import Average,uSIF,sif

In [5]:
english_pings = open(r'All_Agent_English_Pings.txt', encoding='utf-8').readlines()
english_pings = pd.Series(english_pings)
len(english_pings)

1593153

In [6]:
def prep(text):
    data = str(text).lower()
    #to remove data within brackets
    data = re.sub(r'\[.*?\]', '', data) 
    #split the sentences with in a ping and remove punctuations
    data = data.translate(str.maketrans("","",string.punctuation.replace('_','').replace('@',''))).strip() 
    # get rid of numbers
    data = re.sub(r'\d+', '', data)
    # to get rid of all numbers & punctuations
    data = [i for i in data.split() if not any(x in i for x in ['@','http'])]
    # remove words with len <3
    data = [i for i in data if len(i) > 1 or i in ["i"]]
    data = [lemmatizer.lemmatize(i) for i in data]
    return data 

In [7]:
# tqdm._instances.clear()
english_pings_tokens = english_pings.progress_apply(prep)

100%|██████████████████████████████████████████████████████████████████████| 1593153/1593153 [03:49<00:00, 6927.04it/s]


In [8]:
add = ['it','my','zip','code','on','email','to','of','do','me','is','com','1','','u','the','and','can','or','by','up','at','how','are','you']
stop_words =[i for i in set(stopwords.words('english')) if len(i) <2]
stop_words = stop_words + add
stop_words.remove('i')

In [9]:
bigram_phraser_model = Phrases(english_pings_tokens, min_count=3, threshold=2.5)
bigram_features = english_pings_tokens.progress_apply(lambda x: [word for word in bigram_phraser_model[x] if word not in stop_words])

100%|██████████████████████████████████████████████████████████████████████| 1593153/1593153 [03:09<00:00, 8420.62it/s]


In [10]:
def prep_s2v(text):
    data = str(text).lower()
    #to remove data within brackets
    data = re.sub(r'\[.*?\]', '', data) 
    #split the sentences with in a ping and remove punctuations
    data = data.translate(str.maketrans("","",string.punctuation.replace('_','').replace('@',''))).strip() 
    # get rid of numbers
    data = re.sub(r'\d+', '', data)
    # to get rid of all numbers & punctuations
    data = [i for i in data.split() if not any(x in i for x in ['@','http'])]
    # remove words with len <3
    data = [i for i in data if len(i) > 1 or i in ["i"]]
    data = [lemmatizer.lemmatize(i) for i in data]
    #data = [x for x in bigram_phraser_model[data] if x not in stop_words]
    return data

In [11]:
w2v_model = word2vec.Word2Vec(bigram_features, size=100, sg=0, workers=8, min_count=3, window = 5, iter=100)

In [12]:
indexed_sentences = CSplitIndexedList(list(english_pings), custom_split=prep_s2v)
s2v_model = uSIF(w2v_model, workers=2, lang_freq="en")
s2v_model.train(indexed_sentences)

(1593137, 18350476)

# Callback

In [175]:
baselines = pd.read_excel(r"Baselines.xlsx",'callback')
callback_baselines = pd.Series(baselines['Baselines'])
baseline_df = pd.DataFrame()

In [176]:
for sent in tqdm(callback_baselines):
    test_tokens = prep_s2v(sent)
    similar_sentences = s2v_model.sv.similar_by_sentence(test_tokens, model = s2v_model, 
                                                         indexable=indexed_sentences.items, topn=10000)
    similar_sentences = np.array(similar_sentences)
    new_df = pd.DataFrame(similar_sentences[similar_sentences[:,2].astype(float)>=0.4]).sample(10)[[0,2]]
    baseline_df = pd.concat([new_df])
baseline_df.head()

100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [00:13<00:00,  1.60it/s]


,0,2
9712,"after this chat you will be getting a short email survey about your support experience with me. aside from escalating the issue to our relevant team, is there anything else that i can help you with?\n",0.4824163317680359
9228,i have cancelled the subscription and i'll be sending you the email confirmation shortly. is there anything else i can help you with aside from your cancellation request?\n,0.4865388870239258
7230,"our team will definitely notify you via email for any updates about the refund within 5-10 business days, all right?\n",0.5048779249191284
690,bamaarouf hiba: should i escalate it to level 2?\n,0.6299334764480591
3761,is there anything else i can further assist you with aside from following on your request of account deletion and refund request?\n,0.5482227802276611


# Chat End

In [177]:
baselines = pd.read_excel(r"Baselines.xlsx",'chatend')
chatend_baselines = pd.Series(baselines['Baselines'])
baseline_df = pd.DataFrame()

In [178]:
for sent in tqdm(chatend_baselines):
    test_tokens = prep_s2v(sent)
    similar_sentences = s2v_model.sv.similar_by_sentence(test_tokens, model = s2v_model, 
                                                         indexable=indexed_sentences.items, topn=10000)
    similar_sentences = np.array(similar_sentences)
    new_df = pd.DataFrame(similar_sentences[similar_sentences[:,2].astype(float)>=0.4]).sample(10)[[0,2]]
    baseline_df = pd.concat([new_df])
baseline_df.head()

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.51it/s]


,0,2
683,"please be advised that you will receive a feedback within 3-5 business days, okay?\n",0.4597550928592682
1353,you are so welcome! do you have anything else i can help you with? in about six hours you will receive a short survey to your email. i would greatly appreciate it if you could fill that out about our interaction! its helps us with our customer service.\n,0.42468953132629395
218,"also, i wanted to let you know within 4 to 6 hours you should receive a survey through email.. if you could take the time to fill that out, it would be greatly appreciated! is there anything else i can help you with today?\n",0.5080486536026001
2036,have you recived a confirmation email?\n,0.4034988284111023
662,"of course, as you wish, katie. anything else i can do for you? if not, please be aware you'll receive a survey after this chat; we appreciate your feedback!\n",0.46082666516304016


# Greeting

In [179]:
baselines = pd.read_excel(r"Baselines.xlsx",'greeting')
greeting_baselines = pd.Series(baselines['Baselines'])
baseline_df = pd.DataFrame()

In [180]:
for sent in tqdm(greeting_baselines):
    test_tokens = prep_s2v(sent)
    similar_sentences = s2v_model.sv.similar_by_sentence(test_tokens, model = s2v_model, 
                                                         indexable=indexed_sentences.items, topn=10000)
    similar_sentences = np.array(similar_sentences)
    new_df = pd.DataFrame(similar_sentences[similar_sentences[:,2].astype(float)>=0.4]).sample(10)[[0,2]]
    baseline_df = pd.concat([new_df])
baseline_df.head()

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:02<00:00,  2.24it/s]


,0,2
5,"eber jimeno: hello, how are i? i have a client with this email juanluortizcepeda@gmail.com\n",0.463169127702713
51,"hi angel, ho are you?\n",0.41796958446502686
29,hello jeanine. are you with me?\n,0.4308382272720337
9,mardi hind: hello irini how are you?\n,0.4520649313926697
34,"hey gio, are you with me?\n",0.4267681837081909


# Rude

In [181]:
baselines = pd.read_excel(r"Baselines.xlsx",'rude')
rude_baselines = pd.Series(baselines['Baselines'])
baseline_df = pd.DataFrame()

In [182]:
for sent in tqdm(rude_baselines):
    test_tokens = prep_s2v(sent)
    similar_sentences = s2v_model.sv.similar_by_sentence(test_tokens, model = s2v_model, 
                                                         indexable=indexed_sentences.items, topn=10000)
    similar_sentences = np.array(similar_sentences)
    new_df = pd.DataFrame(similar_sentences[similar_sentences[:,2].astype(float)>=0.4]).sample(10)[[0,2]]
    baseline_df = pd.concat([new_df])
baseline_df.head()

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:02<00:00,  2.01it/s]


,0,2
111,can you try getting back in without your wifi connection?\n,0.5092500448226929
210,have you tried to login without using vpn?\n,0.4755154848098755
461,is there someone using your account without your knowledge?\n,0.43550848960876465
621,"just to be sure i met your needs, you mentioned you were unable to stream being that the disney plus content kept buffering, i had you troubleshoot your tv by restating it. afterwards you mentioned you tried another content and it worked. am i correct?\n",0.4208824038505554
693,has she been able to get on her phone with data?\n,0.41588860750198364


In [21]:
import pickle
with open(r's2v_usif_model_dict.pkl', 'wb') as f:
    pickle.dump(s2v_usif_model_dict, f, pickle.HIGHEST_PROTOCOL)